# Lab on Algebraic Structures, Cryptography, and Information Security


In [2]:
import random
import math


## 1. Calculation of Euler's and Möbius functions. Finding the least common multiple of a set of numbers.


### Euler's totient function:
$$ \phi(n) = n \prod_{p|n} \left(1 - \frac{1}{p}\right) $$


In [3]:
def phi(n):
    result = n
    p = 2
    while p * p <= n:
        if n % p == 0:
            while n % p == 0:
                n //= p
            result -= result // p
        p += 1
    if n > 1:
        result -= result // n
    return result

In [4]:
n = 732041
print(phi(n))

732040


### Möbius function:
$$\mu(n) = \begin{cases}
1 & \text{if } n \text { is a square-free positive integer with an even number of prime factors} \\
-1 & \text {if } n \text { is a square-free positive integer with an odd number of prime factors.} \\
0 & \text{otherwise}
\end{cases}$$

In [5]:
def mu(n):
    if n == 1:
        return 1
    result = 1
    p = 2
    while n > 1:
        if n % p == 0:
            n //= p
            if n % p == 0:
                return 0
            result = -result
        p += 1
    return result

In [6]:
n = 105
print(mu(n))

-1



### Least common multiple:
$$
\text{LCM}_n(a_1, a_2, \ldots, a_n) = \text{LCM}(a_1, {LCM}(a_2, {LCM}(a_3, \ldots)))
$$

$$
\text{LCM}(a, b) = \frac{{a \cdot b}}{{\text{gcd}(a, b)}}
$$

$$
\text{gcd}(a, b) = \begin{cases}
a, & \text{if } b = 0 \\
\text{gcd}(b, a\mod b), & \text{otherwise}
\end{cases}
$$

In [7]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def lcm(a, b):
    return a * b // gcd(a, b)

def multi_lcm(numbers):
    result = numbers[0]
    for num in numbers[1:]:
        result = lcm(result, num)

    return result

In [8]:
numbers = [100, 200, 10, 30]
print(multi_lcm(numbers))

600


## 2. Solving a system of linear congruences (Chinese Remainder Theorem).
$$
x \equiv b_1 \pmod{m_1}
$$
$$
x \equiv b_2 \pmod{m_2}
$$
$$ \ldots $$

$$
x \equiv b_n \pmod{m_n}
$$

In [9]:
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        d, x, y = extended_gcd(b % a, a)
        return d, y - (b // a) * x, x

def chinese_remainder(rems, mods):
    modulus = 1
    for mods_i in mods:
        modulus *= mods_i

    multipliers = []
    for i in range(len(mods)):
        N = modulus // mods[i]
        gcd, inverse, y = extended_gcd(N, mods[i])
        multipliers.append(inverse * N % modulus)

    result = 0
    for multi, rems_i in zip(multipliers, rems):
        result = (result + multi * rems_i) % modulus
    return result

In [10]:
mods = [3, 5, 7]
rems = [2, 3, 2]
result = chinese_remainder(rems, mods)
print(result)

23


## 3. Calculation of Legendre symbols and Jacobi symbols

### Calculation of Legendre symbols:
$$
\left(\frac{a}{p}\right) =
\begin{cases}
0, & \text{if } a \equiv 0 \pmod{p} \\
1, & \text{if } a \text{ is a quadratic residue modulo } p \\
-1, & \text{if } a \text{ is a non-quadratic residue modulo } p \\
\end{cases}
$$

In [11]:
def legendre_symbol(a, p):
    if a % p == 0:
        return 0 
    elif pow(a, (p - 1) // 2, p) == 1:
        return 1
    else:
        return -1 

In [12]:
a, p = 13, 7
print(legendre_symbol(a, p))

-1


### Calculation of Jacobi symbols:
$$ (\frac{a}{n}) = (\frac{a}{p_1}) \cdot (\frac{a}{p_2}) \cdot (\frac{a}{p_3}) \cdot \ldots \cdot (\frac{a}{p_k}) $$
$$\text{where } (\frac{a}{p_i}) \text{ is a legendre symbol} $$

In [13]:
def jacobi_symbol(a, n):
    if n <= 0 or n % 2 == 0:
        raise ValueError("n must be a positive odd integer")
    
    a %= n
    t = 1

    while a != 0:
        while a % 2 == 0:
            a /= 2
            r = n % 8
            if r == 3 or r == 5:
                t = -t

        a, n = n, a

        if a % 4 == n % 4 == 3:
            t = -t

        a %= n

    if n == 1:
        return t
    else:
        return 0

In [14]:
a = 1001
n = 9907
print(jacobi_symbol(a, n))

-1


## 4. Pollard Rho algorithm for integer factorization
### Pollarg Rho algorithm:
$$
\begin{align*}
1. & \text{Choose a random starting point } x \text{ and define two functions:} \\
   & \quad f(x) = x^2 + 1 \\
   & \quad g(x) = (x^2 + 1) \mod n, \text{ where } n \text{ is the integer to factor.} \\
2. & \text{Initialize two values, } y \text{ and } d, \text{ to 1.} \\
3. & \text{Repeatedly apply the functions } f \text{ and } g \text{ to } x \text{ while updating } y \text{ and } d \text{ as follows:} \\
   & \quad \text{Update } x \text{ as } x = f(x) \text{ and } y \text{ as } y = g(f(x)). \\
   & \quad \text{Calculate } d \text{ as the greatest common divisor (GCD) of the absolute difference between } x \text{ and } y \text{ and } n, \text{i.e., } d = \text{gcd}(|x - y|, n). \\
4. & \text{Continue this process until } d \text{ is not equal to 1, which means you've found a non-trivial factor of } n. \\
\end{align*}
$$

In [15]:
def f(x):
        return (x**2 + 1) % n

def pollard_rho(n):

    x = random.randint(1, n - 1)
    y = x
    d = 1

    while d == 1:
        x = f(x)
        y = f(f(y))
        d = math.gcd(abs(x - y), n)

    if d == n:
        return None
    else:
        return d

def factorize(n):
    factors = []
    while n > 1:
        if n % 2 == 0:
            factors.append(2)
            n //= 2
        else:
            factor = pollard_rho(n)
            if factor is not None:
                factors.append(factor)
                n //= factor
            else:
                factors.append(n)
                break

    return factors

In [16]:
N = 32498432801
factors = factorize(N)
print(factors)

[43, 755777507]


## 5. Algorithm for finding the discrete logarithm: "Baby-Step Giant-Step" method
$$
\begin{align*}
1. & \text{Choose a base } \alpha, \text{ a target } \beta, \text{ and a prime modulus } p. \\
2. & \text{Compute the value of } m = \lceil \sqrt{p} \rceil. \\
3. & \text{Create two tables:} \\
   & \quad \text{a. Baby table with } i = 0, 1, \ldots, m-1 \text{ where } \text{baby}[i] = \beta \cdot \alpha^{-i} \mod p. \\
   & \quad \text{b. Giant table with } i = 0, 1, \ldots, m-1 \text{ where } \text{giant}[i] = \alpha^{im} \mod p. \\
4. & \text{For each } j = 0, 1, \ldots, m-1: \\
   & \quad \text{a. Check if } \text{baby}[j] \text{ is in the giant table.} \\
   & \quad \text{b. If found, compute the discrete logarithm } x = jm + \text{index of } \text{baby}[j] \text{ in the giant table.} \\
5. & \text{If a match is found, the discrete logarithm is } x. \text{ Otherwise, it does not exist.}
\end{align*}
$$

In [17]:
def discrete_logarithm(a, b, p):
    baby_steps = {}
    m = int(p ** 0.5) + 1
    baby = 1

    for i in range(m):
        baby_steps[baby] = i
        baby = (baby * a) % p

    giant = b
    inv_base_m = pow(a, -m, p)

    for j in range(m):
        if giant in baby_steps:
            return j * m + baby_steps[giant]
        else:
            giant = (giant * inv_base_m) % p

    return None

In [18]:
a = 3
b = 13
p = 17

result = discrete_logarithm(a, b, p)
if result is not None:
    print(result)
else:
    print("No solution found.")

4


## 6. Cipolla's algorithm

$$
\begin{align*}
1. & \text{Compute the Legendre symbol } \left(\frac{a}{p}\right) \text{ as:} \\
   & \quad \left(\frac{a}{p}\right) \equiv a^{(p - 1) / 2} \pmod{p} \\
   & \quad \text{If } \left(\frac{a}{p}\right) \equiv p - 1, \text{ then } a \text{ is a non-quadratic residue modulo } p. \\
2. & \text{If } a \text{ is a non-quadratic residue, no square root exists. Return None.} \\
3. & \text{Determine the factors of } p - 1: p - 1 = 2^s \cdot t. \\
4. & \text{Find a quadratic non-residue modulo } p, \text{ denoted as } z. \\
   & \text{Start with } z = 2 \text{ and increment until } \left(\frac{z}{p}\right) \equiv -1. \\
5. & \text{Calculate } u = a^t \pmod{p} \text{ and } v = a^{(t+1)/2} \pmod{p}. \\
6. & \text{For each } i = 1 \text{ to } s - 1: \\
   & \quad \text{a. Compute } x = u^2 \pmod{p}. \\
   & \quad \text{b. If } x \equiv 1, \text{ break the loop.} \\
   & \quad \text{c. Update } u \text{ as } u = x. \\
   & \quad \text{d. Update } v \text{ as } v = v \cdot z^{2^{s - 1 - i}} \pmod{p}. \\
7. & \text{The square root of } a \text{ modulo } p \text{ is } v. \\
\end{align*}
$$


In [19]:
def find_square_root(a, p):
    if legendre_symbol(a, p) != 1:
        return None
    
    s, t = 0, p - 1
    while t % 2 == 0:
        s += 1
        t //= 2


    z = 2
    while legendre_symbol(z, p) != -1:
        z += 1

    u = pow(a, t, p)
    v = pow(a, (t + 1) // 2, p)

    for i in range(1, s):
        x = pow(u, 2, p)
        if x == 1:
            break
        u = x
        v = (v * pow(z, 2 ** (s - 1 - i), p)) % p

    return v

In [20]:
p, a = 13, 10

result = find_square_root(a, p)
if result is not None:
    print(result)
else:
    print("No square root exists")

9


##  7. Miller-Rabin primality test 
$$
\begin{align*}
1. & \text{Choose a random integer } a \text{ such that } 2 \leq a \leq n - 2. \\
   
2. & \text{Since even numbers other than 2 cannot be prime, we skip them to save computation time.} \\
   
3. & \text{Write } n - 1 \text{ as } 2^r \cdot d, \text{ where } d \text{ is odd.} \\
   
4. & \text{Compute } x \equiv a^d \mod n. \\
   
5. & \text{If } x \equiv 1 \text{ or } x \equiv n - 1, \text{ go to step 8.} \\
   & \text{If } x \equiv 1, \text{return "composite" as } a \text{ is a witness for the composite nature of } n. \\
   & \text{If } x \equiv n - 1, \text{ go to step 8 as } a \text{ may not be a witness for composite.} \\
   
6. & \text{Repeat } r - 1 \text{ times:} \\
   & \quad \text{a. Compute } x \equiv x^2 \mod n. \\
   & \quad \text{b. If } x \equiv n - 1, \text{ go to step 8.} \\
   & \quad \text{c. If } x \equiv 1, \text{return "composite."} \\
   
7. & \text{Return "composite."} \\
   
8. & \text{Repeat steps 1 to 2 for a different random } a \text{ (at most } k \text{ times).} \\
   
9. & \text{Return "probably prime."} \\
\end{align*}
$$



In [21]:
def is_miller_rabin_prime(n, k=10):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

In [22]:
n = 2 ** 127 - 1
if is_miller_rabin_prime(n):
    print(f"{n} is likely a prime number.")
else:
    print(f"{n} is not a prime number.")

170141183460469231731687303715884105727 is likely a prime number.


## 8. RSA (cryptosystem)
$$
\begin{align*}
1. & \text{Key Generation:} \\
   & - \text{Choose two large prime numbers, } p \text{ and } q. \\
   & - \text{Calculate the modulus } n = p \cdot q. \\
   & - \text{Compute } \phi(n) = (p - 1) \cdot (q - 1). \\
   & - \text{Select a public exponent } e \text{ such that } 1 < e < \phi(n) \text{ and } \text{gcd}(e, \phi(n)) = 1. \\
   & - \text{Calculate the private exponent } d \text{ such that } d \equiv e^{-1} \mod \phi(n). \\
   & - \text{The public key is } (n, e) \text{, and the private key is } (n, d). \\
   
2. & \text{Encryption:} \\
   & - \text{Given a plaintext message } M, \text{ convert it into a numerical representation.} \\
   & - \text{Encrypt the message using the public key: } C \equiv M^e \mod n. \\
   
3. & \text{Decryption:} \\
   & - \text{Given the ciphertext } C, \text{ decrypt it using the private key: } M \equiv C^d \mod n. \\
   & - \text{Convert the numerical representation back to the original plaintext.}
\end{align*}
$$

In [23]:
from sympy import mod_inverse

def generate_keypair(bits=512):
    p = generate_prime(bits)
    q = generate_prime(bits)
    n = p * q
    phi = (p - 1) * (q - 1)
    e = select_public_exponent(phi)
    d = mod_inverse(e, phi)
    return ((n, e), (n, d))

def generate_prime(bits):
    while True:
        potential_prime = random.getrandbits(bits)
        if is_miller_rabin_prime(potential_prime, 20):
            return potential_prime

def select_public_exponent(phi):
    e = random.randint(2, phi - 1)
    while gcd(e, phi) != 1:
        e = random.randint(2, phi - 1)
    return e

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def encrypt(public_key, plaintext):
    n, e = public_key
    encrypted_msg = [pow(ord(char), e, n) for char in plaintext]
    return encrypted_msg

def decrypt(private_key, ciphertext):
    n, d = private_key
    decrypted_msg = [chr(pow(char, d, n)) for char in ciphertext]
    return ''.join(decrypted_msg)

In [27]:
    bits = 1024
    public_key, private_key = generate_keypair(bits)

    message = "Example of text"
    encrypted_message = encrypt(public_key, message)
    decrypted_message = decrypt(private_key, encrypted_message)

    print(f"Original message: {message}")
    print(f"Encrypted message: {encrypted_message}"[:200] + "...")
    print(f"Decrypted message: {decrypted_message}")

Original message: Example of text
Encrypted message: [831209326010413285130512889650800323950298633835529212049954560223000237749872831540183748138103981299265829370727930069864416838928152471777877999942276834118549129479315503866138...
Decrypted message: Example of text


## 9. Elliptic Curve El Gamal Cryptosystem